In [9]:
from collections import namedtuple
import altair as alt
import math
import pandas as pd
import streamlit as st
from transformers import BertModel, BertTokenizer
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
import re
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

In [20]:
def summarize_text(url,model, tokenizer):
    # facebook/bart-large-xsum
    
    # https://arxiv.org/abs/1910.13461
    url = url
    response = requests.get(url)
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.text
    #transformed_text = re.sub(r'[^ ]', r'\\', text)
    max_length = 1024
    truncated_input = text[:max_length]

    print(truncated_input)
    # Transform input tokens 
    inputs = tokenizer.encode(truncated_input, return_tensors="pt")

    # Generate summary
    summary_ids = model.generate(inputs, num_beams=4, max_length=1024, early_stopping=True)

    # Convert summary IDs to text
    summary_text = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    return summary_text

In [10]:
model_name = "facebook/bart-large-xsum" 
# Download pytorch model
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [5]:
url ='https://www.abc.net.au/news/2023-07-07/peter-dutton-david-littleproud-coalition-nuclear-reactors/102574782'

In [21]:
summary = summarize_text(url,model,tokenizer)

Peter Dutton opting for the nuclear option for Australia's energy future - ABC NewsSkip to main contentABC News HomepageSearchLoadingMore from ABCMore from ABCClose menuABCiviewListenABC HomeNewsLocallisteniviewEverydayMoreEditorial PoliciesRead our editorial guiding principlesAccessibilityHelpContact UsAbout the ABCPrivacy PolicyTerms of Use© 2023 ABCJust InWatch LiveVoice ReferendumPoliticsWorldBusinessAnalysisSportScienceHealthArtsFact CheckCoronavirusOtherNews HomeABC News HomepagePeter Dutton opting for the nuclear option for Australia's energy futureSharePeter Dutton opting for the nuclear option for Australia's energy futureBy political reporter Monte BovillPosted 9h ago9 hours agoFri 7 Jul 2023 at 3:29am Peter Dutton wants the government to consider nuclear power.(ABC News: Matt Roberts)Help keep family & friends informed by sharing this articleabc.net.au/news/peter-dutton-david-littleproud-coalition-nuclear-reactors/102574782Link copiedCopy linkShareOpposition Leader Peter Dut

In [23]:
summary

"Peter Dutton opting for the nuclear option for Australia's energy future - ABC News."